In [3]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from bs4 import BeautifulSoup
import warnings 
warnings.filterwarnings("ignore")
from gensim.test.utils import datapath
from gensim import utils
import gensim.models
import gensim.downloader as api

from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn as nn 
import torch.nn.functional as f 
from torch.utils.data import Dataset, DataLoader
from gensim.downloader import load
from sklearn.svm import SVC

from tqdm import tqdm 

tqdm.pandas() #### DO NOT FORGET

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


NVIDIA GeForce RTX 3050 Ti Laptop GPU


## 1) Dataset Generation (5 points)
We will use the Amazon reviews dataset used in HW1. Load the dataset
and build a balanced dataset of 250K reviews along with their ratings (50K
instances per each rating score) through random selection. Create ternary
labels using the ratings. We assume that ratings more than 3 denote positive
1
sentiment (class 1) and rating less than 3 denote negative sentiment (class
2). Reviews with rating 3 are considered to have neutral sentiment (class 3).
You can store your dataset after generation and reuse it to reduce the computational load. For your experiments consider a 80%/20% training/testing
split.

In [4]:
df = pd.read_csv("data/amazon_reviews_us_Office_Products_v1_00.tsv", sep='\t', on_bad_lines='skip')#, usecols=['review_body','star_rating']) #lineterminator='\r'
df.drop(df.columns[0], axis=1, inplace=True)
df = df[['review_body', 'star_rating']]
df.columns

Index(['review_body', 'star_rating'], dtype='object')

### Preprocess/Cleaning take 15 minutes

In [5]:
contraction_mapping = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}


pattern_contractions = re.compile('(%s)' % '|'.join(contraction_mapping.keys()))
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords', 'punkt')
stop_words = set(stopwords.words('english'))


def expand_contractions(text, contraction_map=contraction_mapping):
    return pattern_contractions.sub(lambda occurrence: contraction_map[occurrence.group(0)], text)


def rem_stopwords(review,stp):
    words = review.split()
    filtered_words = [word for word in words if word not in stp]
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence


def lemmazation(review):
    words = review.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_review = ' '.join(lemmatized_words)
    return lemmatized_review


def clean_preproc_reviews(reviews, stp):
    ### CLEANING
    reviews = reviews.str.lower()
    # reviews = reviews.progress_apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
    # reviews = reviews.replace(r'http\S+', '', regex=True)
    # reviews = reviews.replace("[^a-zA-Z]", " ", regex=True)
    # reviews = reviews.replace('\s+', ' ', regex=True).str.strip()
    # reviews = reviews.progress_apply(lambda x: expand_contractions(x))

    ### PREPROCESSING
    # reviews = reviews.progress_apply(lambda x : rem_stopwords(x, stp))
    # reviews = reviews.progress_apply(lemmazation)

    return reviews

# Clean the reviews
df['review_body'] = df['review_body'].astype(str)
df.dropna(subset=['review_body'], inplace=True)
df['review_body'] =  clean_preproc_reviews(df['review_body'], stop_words) #df['review_body'].progress_apply(lambda x: clean_preproc_reviews(x, stop_words) )

df.dropna(subset=['review_body'], inplace=True)


[nltk_data] Downloading package stopwords to punkt...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df['label'] = df['star_rating'].progress_apply(lambda x: 0 if x in [4, 5] else (1 if x in [1, 2] else 2))

star_ratings = [5, 4, 3, 2, 1]
samples = [ df[df['star_rating'] == rating].sample(n = 50000, random_state = 42) for rating in star_ratings]
merged_dataset = pd.concat(samples)

  0%|          | 0/2640254 [00:00<?, ?it/s]

100%|██████████| 2640254/2640254 [00:02<00:00, 1209107.60it/s]


In [7]:
# print("Sample review and label after creating merged dataset:")
# print(merged_dataset[['review_body', 'star_rating', 'label']].sample(1))


## 2) Word Embedding (30 points)
In this part the of the assignment, you will learn how to generate two sets
of Word2Vec features for the dataset you generated. You can use Gensim
library for this purpose. A helpful tutorial is available in the following link:
https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.
html


### (a) (10 points)
Load the pretrained “word2vec-google-news-300” Word2Vec model and learn
how to extract word embeddings for your dataset. Try to check semantic
similarities of the generated vectors using two examples of your own, e.g.,
King − M an + W oman = Queen or excellent ∼ outstanding.


In [8]:
# # wv['buger'] - wv['fries'] + wv['fish'] ?= wv['chips']
# test_relationship = wv['burger'] - wv['fries'] + wv['fish']
# print(test_relationship, wv['chips']) 


In [9]:
pretrained_model = api.load('word2vec-google-news-300')

result = pretrained_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print("King - Man + Woman =", result[0][0])

similarity = pretrained_model.similarity('excellent', 'outstanding')
print("Similarity between 'excellent' and 'outstanding': ", similarity)


King - Man + Woman = queen
Similarity between 'excellent' and 'outstanding': 0.5567486


In [10]:
# # Function to find the most similar word
# def most_similar_vector(vector):
#     return pretrained_model.similar_by_vector(vector, topn=1)[0][0]
# #
# # Vector arithmetic: "King - Man + Woman"
# result_vector = pretrained_model['king'] - pretrained_model['man'] + pretrained_model['woman']
# analogy_result = most_similar_vector(result_vector)
# print(f"King - Man + Woman = {analogy_result}")

# # Function to compute cosine similarity
# def cosine_similarity(vec1, vec2):
#     return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# # Compute similarity between 'excellent' and 'outstanding'
# similarity_score = cosine_similarity(pretrained_model['excellent'], pretrained_model['outstanding'])
# print(f"Similarity between 'excellent' and 'outstanding': {similarity_score}")


### (b) (20 points)
Train a Word2Vec model using your own dataset. Set the embedding size
to be 300 and the window size to be 11. You can also consider a minimum
word count of 10. Check the semantic similarities for the same two examples
in part (a). What do you conclude from comparing vectors generated by
yourself and the pretrained model? Which of the Word2Vec models seems
to encode semantic similarities between words better?

In [11]:
class MyCorpus:
    def __init__(self, df, col):
        self.df = df
        self.col = col

    def __iter__(self):
        for line in self.df[self.col]:
            yield utils.simple_preprocess(line)


my_model = gensim.models.Word2Vec(sentences = MyCorpus(merged_dataset, 'review_body'), vector_size=300, window=11, min_count=10, workers=4)

In [12]:
# ### NEW Dataframe

# word_vectors = my_model.wv

# result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
# print("King - Man + Woman =", result[0][0])
# similarity = word_vectors.similarity('excellent', 'outstanding')
# print("Similarity between 'excellent' and 'outstanding':", similarity)
# print( my_model.wv)

In [13]:
# For the pretrained model
pretrained_vocab_size = len(pretrained_model.key_to_index)
print("Vocabulary size of the pretrained model:", pretrained_vocab_size)

# For your trained model
my_model_vocab_size = len(my_model.wv.key_to_index)
print("Vocabulary size of my model:", my_model_vocab_size)


Vocabulary size of the pretrained model: 3000000
Vocabulary size of my model: 16476


In [14]:
def document_vector(word2vec_model, doc_review):

    doc_review = [word for word in doc_review if word in word2vec_model.key_to_index]

    if len(doc_review) == 0:
        return np.zeros(word2vec_model.vector_size)
        
    return np.mean(word2vec_model[doc_review], axis=0)

def gen_concat_feature_vector(word2vec_model, doc_review, vector_size=300, max_words=10):
    concatenated_vector = np.zeros(vector_size * max_words) # Initialize an empty array for the concatenated vectors.

    for i, word in enumerate(doc_review[:max_words]):
        if word in word2vec_model.key_to_index:
            concatenated_vector[i*vector_size:(i+1)*vector_size] = word2vec_model[word]
            
    return concatenated_vector


merged_dataset['processed_text'] = merged_dataset['review_body'].progress_apply(gensim.utils.simple_preprocess)
merged_dataset['pretrained_vector'] = merged_dataset['processed_text'].progress_apply(lambda doc_review: document_vector(pretrained_model, doc_review))
merged_dataset['custom_vector'] = merged_dataset['processed_text'].progress_apply(lambda doc_review: document_vector(my_model.wv, doc_review))
#####
merged_dataset['pre_concatenated_vector'] = merged_dataset['processed_text'].progress_apply(lambda row_indx: gen_concat_feature_vector(pretrained_model, row_indx))
merged_dataset['custom_concatenated_vector'] = merged_dataset['processed_text'].progress_apply(lambda row_indx: gen_concat_feature_vector(pretrained_model, row_indx))

X = np.vstack( merged_dataset['custom_vector'] )
X_pre = np.vstack( merged_dataset['pretrained_vector']) 
Y = np.vstack( merged_dataset['label'] ) 


  1%|          | 2093/250000 [00:00<00:12, 20103.05it/s]

100%|██████████| 250000/250000 [00:26<00:00, 9555.49it/s] 


In [15]:
# X_custom = np.vstack( merged_dataset['custom_custom_vector'] )
# X_pre = np.vstack( merged_dataset['pretrained_vector']) 
# Y_ter = np.vstack( merged_dataset['label'] ) 
# Y_bin = np.vstack( merged_dataset['binary_label'] ) 

# custom_ter_data = X_custom, Y_ter   # Custom embedding, ternary labeled data 
# pre_ter_data = X_pre, Y_ter         # Pretrained embeddings, ternary labeled data  

# custom_bin_data = X_custom, Y_bin   # Custom embedding, binary labeled data 
# pre_bin_data = X_pre, Y_bin         # Pretrained embeddings, binary labeled data

In [16]:
# merged_dataset['custom_vector'].iloc[1].shape

## 3) Simple models (20 points)
Using the Word2Vec features that you can generate using the two models
you prepared in the Word Embedding section, train a perceptron and an
SVM model similar to HW1 for class 1 and class 2 (binary models). For this
purpose, you can just use the average Word2Vec vectors for each review as
the input feature (x = 1N PNi=1 Wi for a review with N words). To improve 2
your performance, use the data cleaning and preprocessing steps of HW1
to include only important words from each review when you compute the
average x = 1 N PN i=1 Wi.
Report your accuracy values on the testing split for
these models for each feature type along with values you reported in your
HW1 submission, i.e., for each of perceptron and SVM, you need to report
three accuracy values for “word2vec-google-news-300”, your own Word2Vec,
and TF-IDF features.
What do you conclude from comparing performances for the models
trained using the three different feature types (TF-IDF, pretrained Word2Vec,
your trained Word2Vec)?


In [17]:
def evaulate(y_label, y_predicted):
    accuracy = accuracy_score(y_label, y_predicted)
    precision = precision_score(y_label, y_predicted, average='weighted')
    recall = recall_score(y_label, y_predicted, average='weighted')
    f1 = f1_score(y_label, y_predicted, average='weighted')
    return accuracy, precision, recall,f1

In [18]:
def sklearn_model_train(data, model_type):
    for name, model_name in model_type:
        X_train, X_test, y_train, y_test = data 
        # clf = model_name()
        model_name.fit(X_train, y_train)
        y_pred_train = model_name.predict(X_train)
        y_pred_test = model_name.predict(X_test)

        tr_acc, tr_prec, tr_rec, tr_f1 = evaulate(y_train, y_pred_train)
        te_acc, te_prec, te_rec, te_f1 = evaulate(y_test, y_pred_test)

        print("Training: Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(tr_acc, tr_prec, tr_rec, tr_f1))
        print(" Testing: Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(te_acc, te_prec, te_rec, te_f1) )

split_data = train_test_split(X, Y, test_size=0.2, random_state=42)
pretrain_split_data = train_test_split(X_pre, Y, test_size=0.2, random_state=42)

model_names = [
    ("Perceptron", Perceptron())#,
    #("SVC", SVC())
]

sklearn_model_train(split_data, model_names)
sklearn_model_train(pretrain_split_data, model_names)


Training: Accuracy: 0.5941, Precision: 0.6244, Recall: 0.5941, F1-Score: 0.6005
 Testing: Accuracy: 0.5919, Precision: 0.6226, Recall: 0.5919, F1-Score: 0.5984
Training: Accuracy: 0.5893, Precision: 0.6136, Recall: 0.5893, F1-Score: 0.5488
 Testing: Accuracy: 0.5870, Precision: 0.6125, Recall: 0.5870, F1-Score: 0.5471


## 4) Feedforward Neural Networks (25 points)
Using the features that you can generate using the models you prepared in
the Word “Embedding section”, train a feedforward multilayer perceptron
network for sentiment analysis classification. Consider a network with two
hidden layers, each with 50 and 10 nodes, respectively. You can use cross
entropy loss and your own choice for other hyperparamters, e.g., nonlinearity,
number of epochs, etc. Part of getting good results is to select good values
for these hyperparamters.
You can also refer to the following tutorial to familiarize yourself:
https://www.kaggle.com/mishra1993/pytorch-multi-layer-perceptron-mnist
Although the above tutorial is for image data but the concept of training
an MLP is very similar to what we want to do.

### (a) (10 points)
To generate the input features, use the average Word2Vec vectors similar to
the “Simple models” section and train the neural network. Train a network
for binary classification using class 1 and class 2 and also a ternary model for
the three classes. Report accuracy values on the testing split for your MLP
model for each of the binary and ternary classification cases.


## TERNARY

In [19]:
class Net(nn.Module): 
    def __init__(self, n_classes, n_dim):
        super(Net, self).__init__()

        hidden_1 = 50
        hidden_2 = 10

        self.n_classes = n_classes
        self.n_dim = n_dim

        self.fc1 = nn.Linear(n_dim, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, n_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = f.gelu(self.fc1(x))
        x = self.dropout(x)
        x = f.gelu(self.fc2(x))
        x = self.dropout(x)
        x = f.softmax(self.fc3(x)) # REMOVE SOFTMAX?
        return x 


ternary_model = Net(n_classes = 3, n_dim = 300)
binary_model = Net(n_classes = 2, n_dim = 300)
concat_ternary_model = Net(n_classes = 3, n_dim = 3000)
concat_binary_model = Net(n_classes = 2, n_dim = 3000)

print(ternary_model)

Net(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [20]:
# print("Features and labels before creating datasets:")
# for i in range(5):  
#     vector_norm = torch.norm(X_train_tensor[i], p=2)  # L2 norm 
#     #vector_summary = ", ".join(f"{x:.2f}" for x in X_train_tensor[i][:3]) + "..."  # Summarize vector 
#     print(f"Sample {i+1}: Norm = {vector_norm:.2f}, Label: {y_train_tensor[i]}")

In [21]:
class TextDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
x_train = torch.tensor(np.array(x_train.tolist(), dtype=np.float32), dtype=torch.float32)
    y_train = torch.tensor(np.array(y_train), dtype=torch.long)
    x_test = torch.tensor(np.array(x_test.tolist(), dtype=np.float32), dtype=torch.float32)
    y_test = torch.tensor(np.array(y_test), dtype=torch.long)

In [118]:
def model_preprocess(x, y, model, cnn_bit:int = 0, optimi=None):
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    x_train = torch.tensor(np.array(x_train.tolist(), dtype=np.float32), dtype=torch.float32)
    y_train = torch.tensor(np.array(y_train), dtype=torch.long)
    x_test = torch.tensor(np.array(x_test.tolist(), dtype=np.float32), dtype=torch.float32)
    y_test = torch.tensor(np.array(y_test), dtype=torch.long)

    train_loader = DataLoader(TextDataset(x_train, y_train), batch_size=4, shuffle=True)
    test_loader = DataLoader(TextDataset(x_test, y_test), batch_size=4, shuffle=False)
    
    if optimi is None:
        optimi = torch.optim.Adadelta(model.parameters(), lr=0.25, rho=0.95) if cnn_bit == 1 else torch.optim.Adam(model.parameters(), lr=0.0001)
        print(optimi)

    return (train_loader, test_loader, nn.CrossEntropyLoss(), optimi)


In [106]:
def train_model(hyperparams, model, n_epochs = 5):
    # model.to(device)
    
    train_loader, test_loader, criterion, optimizer = hyperparams
    valid_loss_min = np.Inf
    
    for epoch in range(n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        correct_train = 0
        correct_valid = 0

        model.train()
        accumulation_steps = 4  # Accumulate gradients over 4 forward passes

        for i, (data, target) in enumerate(train_loader):
            # data, target = data.to(device), target.to(device)
            if i % accumulation_steps == 0:
                optimizer.zero_grad()

            output = model(data)
            target = target.squeeze()
            # print(output.shape, "torch.argmax: ", torch.argmax(output, dim=0),"torch.max: ", torch.max(output.data, 1) )
            loss = criterion(output, target)
            loss.backward()
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                optimizer.step()
                
            train_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output.data, 1)
            correct_train += (predicted == target).sum().item()

        model.eval()
        with torch.no_grad():
            for data, target in test_loader:
                # data, target = data.to(device), target.to(device)
                output = model(data)
                target = target.squeeze()
                loss = criterion(output, target)
                valid_loss += loss.item() * data.size(0)
                _, predicted = torch.max(output.data, 1)
                correct_valid += (predicted == target).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        valid_loss = valid_loss / len(test_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        valid_accuracy = correct_valid / len(test_loader.dataset)

        print(f'Epoch: {epoch+1} \tTraining Loss: {train_loss:.6f} \tTraining Accuracy: {train_accuracy * 100:.2f}% \tValidation Loss: {valid_loss:.6f} \tValidation Accuracy: {valid_accuracy * 100:.2f}%')

        if valid_loss < valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...')
            torch.save(model.state_dict(), 'model.pt')
            valid_loss_min = valid_loss

In [24]:
# model_hyperparameters_tern_custom = model_preprocess(merged_dataset['custom_vector'].values, merged_dataset['label'].values, ternary_model)
# train_model(model_hyperparameters_tern_custom, ternary_model)

In [25]:
# model_hyperparameters_tern_pre = model_preprocess(merged_dataset['pretrained_vector'].values, merged_dataset['label'].values, ternary_model)
# train_model(model_hyperparameters_tern_pre, ternary_model)

In [26]:
# merged_dataset['binary_label'] = merged_dataset['label'].progress_apply(lambda x: pd.NA if x == 2 else (0 if x == 0 else 1))
filtered_dataset = merged_dataset[merged_dataset['label'] != 2]
filtered_dataset['binary_label'] = filtered_dataset['label'].astype(int)

# model_hyperparameters_bin_custom = model_preprocess(filtered_dataset['custom_vector'].values, filtered_dataset['binary_label'].values, binary_model)
# train_model(model_hyperparameters_bin_custom, binary_model)

In [27]:
# model_hyperparameters_bin_pre = model_preprocess(filtered_dataset['pretrained_vector'].values, filtered_dataset['binary_label'].values, binary_model)
# train_model(model_hyperparameters_bin_pre, binary_model)

(b) (15 points)
To generate the input features, concatenate the first 10 Word2Vec vectors
for each review as the input feature (x = [WT
1, ..., WT [10]) and train the neural 3
network. Report the accuracy value on the testing split for your MLP model
for each of the binary and ternary classification cases.
What do you conclude by comparing accuracy values you obtain with
those obtained in the “’Simple Models” section (note you can compare the
accuracy values for binary classification).

In [28]:
# model_hyperp_concat_bin_pre = model_preprocess(filtered_dataset['pre_concatenated_vector'].values, filtered_dataset['binary_label'].values, concat_binary_model)
# train_model(model_hyperp_concat_bin_pre, concat_binary_model, n_epochs = 5)

In [29]:
# model_hyperp_concat_bin_pre = model_preprocess(filtered_dataset['custom_concatenated_vector'].values, filtered_dataset['binary_label'].values, concat_binary_model)
# train_model(model_hyperp_concat_bin_pre, concat_binary_model, n_epochs = 5)

In [30]:
# model_hyperp_concat_ter_pre = model_preprocess(merged_dataset['pre_concatenated_vector'].values, merged_dataset['label'].values, concat_ternary_model)
# train_model(model_hyperp_concat_ter_pre, concat_ternary_model, n_epochs = 5)

In [31]:
# model_hyperp_concat_ter_pre = model_preprocess(merged_dataset['pre_concatenated_vector'].values, merged_dataset['label'].values, concat_ternary_model)
# train_model(model_hyperp_concat_ter_pre, concat_ternary_model, n_epochs = 5)

## 5) Convolutional Neural Networks (20 points)
Using the vectors you prepared in the “Word Embedding” section, train a
convolutional neural network (CNN) for sentiment analysis classification.
Train a simple CNN for sentiment analysis. You can consider an two-layer
CNN with the output channel sizes of 50 and 10. To feed your data into the
CNN, limit the maximum review length to 50 by truncating longer reviews
and padding shorter reviews with a null value (0). You can use cross entropy
loss and your own choice for other hyperparamters, e.g., nonlinearity, number
of epochs, etc. Train the CNN network for binary classification using class 1
and class 2 and also a ternary model for the three classes. Report accuracy
values on the testing split for your CNN model.

In [116]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Initialization code remains unchanged
        self.num_classes = num_classes
        self.conv1 = nn.Conv1d(1, 50, kernel_size = 5, padding = 2) ## 50 words store as 300 dim wv's 
        self.conv2 = nn.Conv1d(50, 10, kernel_size = 5, padding = 2) 
        self.fc = nn.Linear(3000, self.num_classes)
        self.optimizer = torch.optim.Adadelta(model.parameters(), lr=0.25, rho=0.95) 
        
    def forward(self, x):
        # Convert input to long tensor
        x = x.squeeze()  # Ensure the input indices are of type Long
        # x = self.embedding(x).permute(0, 2, 1)
        x = x.reshape(x.shape[0], 1 , x.shape[1])
        x = F.gelu(self.conv1(x))
        x = F.gelu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



### Custom & 3 Class

In [119]:
CNNnet_ter_custom = SimpleCNN(num_classes = 3)
cnn_hyperp_ter = model_preprocess(x = merged_dataset['custom_vector'].values, y = merged_dataset['label'].values, model = CNNnet_ter_custom, cnn_bit = 1)
train_model(hyperparams = cnn_hyperp_ter, model = CNNnet_ter_custom )

Adadelta (
Parameter Group 0
    differentiable: False
    eps: 1e-06
    foreach: None
    lr: 0.25
    maximize: False
    rho: 0.95
    weight_decay: 0
)


### Custom & 2 Class

### Pretrained & 2 Class

### Pretrained & 3 Class